In [13]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import tensorflow as tf
from tensorflow import keras
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection, naive_bayes, svm
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.decomposition import TruncatedSVD
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GlobalMaxPooling1D, Conv1D, Flatten
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

!pip3 install tensorflow_hub
import tensorflow_hub as hub

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from keras import layers
import sys
import os

In [14]:
scriptpath = "../scripts/"

# Add the directory containing your module to the Python path (wants absolute paths)
sys.path.append(os.path.abspath(scriptpath))

In [17]:
sys.path

['/home/ragou/repo/github/RAPMC/notebook',
 '',
 '/home/ragou/.local/lib/python3.8/site-packages',
 '/usr/lib/python38.zip',
 '/usr/lib/python3.8',
 '/usr/lib/python3.8/lib-dynload',
 '/usr/local/lib/python3.8/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/lib/python3.8/dist-packages',
 '/home/ragou/.local/lib/python3.8/site-packages/IPython/extensions',
 '/home/ragou/.ipython',
 '/home/ragou/repo/github/RAPMC/scripts']

In [22]:
!chmod 775 ../scripts/article_cleaner.py
import article_cleaner

# Load test data

In [24]:
test_variant = "../../data/test_variants/test_variants"
test_text = "../../data/test_text/test_text"

In [31]:
dtf_text = pd.read_csv(test_text, sep = "\|\|")

In [33]:
dtf_text.columns = ["Text"]
dtf_text.index.name = "ID"

In [34]:
dtf_text

,Text
ID,
0,2. This mutation resulted in a myeloproliferat...
1,Abstract The Large Tumor Suppressor 1 (LATS1)...
2,Vascular endothelial growth factor receptor (V...
3,Inflammatory myofibroblastic tumor (IMT) is a ...
4,Abstract Retinoblastoma is a pediatric retina...
...,...
5663,The realization in the late 1970s that RAS har...
5664,Hemizygous deletions are common molecular abno...
5665,All most R267W of has with to SMARTpool invest...


In [37]:
dtf_var = pd.read_csv(test_variant)

In [38]:
dtf_var

,ID,Gene,Variation
0,0,ACSL4,R570S
1,1,NAGLU,P521L
2,2,PAH,L333F
3,3,ING1,A148D
4,4,TMEM216,G77A
...,...,...,...
5663,5663,SLC46A1,R113S
5664,5664,FOXC1,L130F
5665,5665,GSS,R267W
5666,5666,CTSK,G79E


In [40]:
dtf_concat = pd.merge(dtf_var, dtf_text, on = "ID")

In [49]:
dtf_concat["Gene"]

0         ACSL4
1         NAGLU
2           PAH
3          ING1
4       TMEM216
         ...   
5663    SLC46A1
5664      FOXC1
5665        GSS
5666       CTSK
5667     DFNB59
Name: Gene, Length: 5668, dtype: object

In [ ]:
clean_test = dtf_concat.apply(lambda line : article_cleaner.extract_match(line), axis = 1)

# Load model CNN

In [2]:
model = keras.models.load_model('model_cnn.h5')

In [6]:
model.weights

[<tf.Variable 'embedding_28/embeddings:0' shape=(2000, 128) dtype=float32, numpy=
 array([[-0.01194105, -0.00893595, -0.09128901, ..., -0.01825847,
          0.00260322, -0.0536463 ],
        [ 0.14629889,  0.0564375 , -0.11996924, ...,  0.0667411 ,
          0.11231963, -0.11683479],
        [ 0.04504862,  0.1216522 ,  0.08654026, ...,  0.07509339,
          0.0276182 , -0.08756479],
        ...,
        [ 0.07698343, -0.00126461, -0.03485166, ..., -0.09692236,
         -0.01976166, -0.05523314],
        [-0.03065643,  0.04007576,  0.00551316, ..., -0.02421315,
         -0.09088104,  0.05043201],
        [ 0.04888308, -0.00193551, -0.00225104, ...,  0.00101527,
          0.01284167,  0.03524701]], dtype=float32)>,
 <tf.Variable 'conv1d_48/kernel:0' shape=(5, 128, 124) dtype=float32, numpy=
 array([[[ 0.11764958,  0.05362569, -0.04365219, ..., -0.07531936,
          -0.05758185, -0.01075119],
         [ 0.06658083, -0.06791958,  0.08262946, ..., -0.14793019,
          -0.04753087, -0.0